In [1]:
!pip install scikit-optimize
!pip install -U pymoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210079 sha256=090f48bbebbdf0f027cd924959a437570494fb4309c863f694fe48ca5e9ea968
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully buil

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from skopt import gp_minimize
import matplotlib.pyplot as plt

In [3]:
# Carregar os dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

train_mask = np.logical_or(y_train == 0, y_train == 1)
test_mask = np.logical_or(y_test == 0, y_test == 1)
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

# Normalizar os dados
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
def build_cnn_model():
    # Criar o modelo da rede neural convolucional
    model = keras.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(2, activation="softmax")
    ])

    return model

In [5]:
def chebyshev_loss(y_true, y_pred, model, lambda1, lambda2):
    # Função de perda Categorical Crossentropy
    ce_loss = keras.losses.CategoricalCrossentropy()(y_true, y_pred)

    # Função de regularização L2
    l2_loss = sum([tf.nn.l2_loss(layer.kernel) for layer in model.layers if 'Conv2D' in str(layer) or 'Dense' in str(layer)])

    # Combinar as funções de perda com os pesos de Chebyshev
    combined_loss = lambda1 * ce_loss + lambda2 * l2_loss

    return combined_loss

In [6]:
def objective(params):
    lambda1, lambda2 = params
    
    # Construir o modelo da CNN
    model = build_cnn_model()

    # Compilar o modelo com a função de perda personalizada e otimizador
    model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, model, lambda1, lambda2), optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

    # Treinar o modelo
    model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

    # Avaliar o desempenho do modelo
    score = model.evaluate(x_test, y_test, verbose=0)
    
    return score[0]

In [7]:
space = [(0.0001, 0.01, "uniform"), (0.0001, 0.01, "uniform")]

result = gp_minimize(func=objective, dimensions=space, n_calls=10, random_state=42, verbose=1)

#Imprimir os melhores hiperparâmetros encontrados
best_lambda1, best_lambda2 = result.x
print(f"Melhores hiperparâmetros encontrados: lambda1 = {best_lambda1}, lambda2 = {best_lambda2}")

Iteration No: 1 started. Evaluating function at random point.
Epoch 1/2
90/90 [==============================] - 19s 200ms/step - loss: 0.0602 - accuracy: 0.9855 - val_loss: 0.0045 - val_accuracy: 0.9890
Epoch 2/2
90/90 [==============================] - 13s 144ms/step - loss: 0.0035 - accuracy: 0.9924 - val_loss: 0.0031 - val_accuracy: 0.9890
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 33.5091
Function value obtained: 0.0030
Current minimum: 0.0030
Iteration No: 2 started. Evaluating function at random point.
Epoch 1/2
90/90 [==============================] - 11s 114ms/step - loss: 0.1805 - accuracy: 0.7804 - val_loss: 0.0107 - val_accuracy: 0.5162
Epoch 2/2
90/90 [==============================] - 10s 112ms/step - loss: 0.0075 - accuracy: 0.5341 - val_loss: 0.0061 - val_accuracy: 0.5162
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 22.1738
Function value obtained: 0.0060
Current minimum: 0.0030
Iteration No: 3 started. Evaluating function

In [8]:
#Construir e treinar o melhor base
base_model = build_cnn_model()
base_model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
base_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

#Avaliar o melhor base
score = base_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

Epoch 1/2
90/90 [==============================] - 10s 97ms/step - loss: 0.0344 - accuracy: 0.9913 - val_loss: 0.0014 - val_accuracy: 0.9992
Epoch 2/2
67/67 [==============================] - 1s 9ms/step - loss: 0.0030 - accuracy: 0.9995
Test loss: 0.0029672093223780394, Test accuracy: 0.9995272159576416


In [9]:
#Construir e treinar o melhor modelo
best_model = build_cnn_model()
best_model.compile(loss=lambda y_true, y_pred: chebyshev_loss(y_true, y_pred, best_model, best_lambda1, best_lambda2),
optimizer=keras.optimizers.Adam(), metrics=["accuracy"])
best_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1, verbose=1)

#Avaliar o melhor modelo
score = best_model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {score[0]}, Test accuracy: {score[1]}")

Epoch 1/2
90/90 [==============================] - 12s 115ms/step - loss: 0.0072 - accuracy: 0.9891 - val_loss: 8.8209e-04 - val_accuracy: 0.9976
Epoch 2/2
67/67 [==============================] - 1s 9ms/step - loss: 4.5335e-04 - accuracy: 0.9991
Test loss: 0.00045335464528761804, Test accuracy: 0.9990543723106384
